# Segmenting and Clustering Toronto Neighborhoods

Load dependencies.

In [1]:
import pandas as pd
import numpy as np

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

#!conda install -c conda-forge folium
import folium # map rendering library

# import k-means from clustering stage
from sklearn.cluster import KMeans

## 1. Download and Explore Data

Scrape Toronto postal code data from the web and create a dataframe.

In [2]:
web_tables = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
postal_codes = web_tables[0]
postal_codes.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [3]:
postal_codes.shape

(180, 3)

Remove the "Not assigned" boroughs.

In [4]:
boroughs = postal_codes[postal_codes["Borough"] != "Not assigned"].reset_index(drop = True)
boroughs.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Check if any values are still missing.

In [5]:
boroughs.isnull().values.any()

False

Check out basic counts and shape of dataframe.

In [6]:
boroughs.describe()

,Postal Code,Borough,Neighborhood
count,103,103,103
unique,103,10,99
top,M2K,North York,Downsview
freq,1,24,4


In [7]:
boroughs.shape

(103, 3)

#### Find the latitude and longitude of each postal code using the provided .csv file because geocoder was unreliable.

In [8]:
lat_long_df = pd.read_csv("https://cocl.us/Geospatial_data")
lat_long_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [9]:
lat_long_df.shape

(103, 3)

In [10]:
boroughs["Latitude"] = ""
boroughs["Longitude"] = ""
boroughs.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,,
1,M4A,North York,Victoria Village,,
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",,
3,M6A,North York,"Lawrence Manor, Lawrence Heights",,
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",,


Merge dataframes - a not very fancy nested for-loop should do the trick.

In [11]:
for index,row in lat_long_df.iterrows():
    for i, j in boroughs.iterrows():
        if lat_long_df.at[index, "Postal Code"] == boroughs.at[i, "Postal Code"]:
            boroughs.at[i, "Latitude"] = lat_long_df.at[index, "Latitude"]
            boroughs.at[i, "Longitude"] = lat_long_df.at[index, "Longitude"]
            break
            
boroughs.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7533,-79.3297
1,M4A,North York,Victoria Village,43.7259,-79.3156
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6543,-79.3606
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7185,-79.4648
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6623,-79.3895


In [12]:
boroughs.shape

(103, 5)

Display all the postal codes and list their boroughs / neighborhoods.

In [13]:
#Toronto coordinates
latitude = 43.653316
longitude = -79.384100

# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(boroughs['Latitude'], boroughs['Longitude'], boroughs['Borough'], boroughs['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Let's focus on Downtown Toronto.

In [14]:
downtown_data = boroughs[boroughs["Borough"] == "Downtown Toronto"].reset_index(drop=True)
downtown_data

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6543,-79.3606
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6623,-79.3895
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.6572,-79.3789
3,M5C,Downtown Toronto,St. James Town,43.6515,-79.3754
4,M5E,Downtown Toronto,Berczy Park,43.6448,-79.3733
5,M5G,Downtown Toronto,Central Bay Street,43.658,-79.3874
6,M6G,Downtown Toronto,Christie,43.6695,-79.4226
7,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.6506,-79.3846
8,M5J,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands",43.6408,-79.3818
9,M5K,Downtown Toronto,"Toronto Dominion Centre, Design Exchange",43.6472,-79.3816


Let's visualize Downtown Toronoto by postal code / neighborhood.

In [15]:
# create map of Toronto using the latitude and longitude values defined for the greater city (above)
map_downtown_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(downtown_data['Latitude'], downtown_data['Longitude'], downtown_data['Borough'], downtown_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_downtown_toronto)  

map_downtown_toronto

## 2. Explore Downtown Venues

Use the Foursquare API to explore the neighborhoods and segment them.

In [36]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 
CLIENT_SECRET:


Create a function to collect venues within a specific area.

In [17]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Now, let's get the top 100 venues that are in Parkwoods within a radius of 500 meters.

In [18]:
downtown_venues = getNearbyVenues(names = downtown_data['Neighborhood'],
                                   latitudes = downtown_data['Latitude'],
                                   longitudes = downtown_data['Longitude']
                                  )


Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Stn A PO Boxes
St. James Town, Cabbagetown
First Canadian Place, Underground city
Church and Wellesley


In [19]:
print (downtown_venues.shape)
downtown_venues.head()

(1214, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
3,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
4,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


Let's check how many venues were returned for each neighborhood.

In [20]:
downtown_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,57,57,57,57,57,57
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",15,15,15,15,15,15
Central Bay Street,64,64,64,64,64,64
Christie,17,17,17,17,17,17
Church and Wellesley,77,77,77,77,77,77
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
"First Canadian Place, Underground city",100,100,100,100,100,100
"Garden District, Ryerson",100,100,100,100,100,100
"Harbourfront East, Union Station, Toronto Islands",100,100,100,100,100,100


Let's find out how many unique categories can be curated from all the returned venues.

In [21]:
print('There are {} uniques categories.'.format(len(downtown_venues['Venue Category'].unique())))

There are 212 uniques categories.


## 3. Analyze Venues in Each Neighborhood

First, create binary values for each venue category.

In [22]:
# one hot encoding
downtown_onehot = pd.get_dummies(downtown_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
downtown_onehot['Neighborhood'] = downtown_venues['Neighborhood'] 

# move neighborhood column to the first column
first_column = downtown_onehot.pop("Neighborhood")
downtown_onehot.insert(0, "Neighborhood", first_column)

downtown_onehot.head()

,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


And let's examine the new dataframe size.

In [23]:
downtown_onehot.shape

(1214, 212)

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [24]:
downtown_grouped = downtown_onehot.groupby('Neighborhood').mean().reset_index()
downtown_grouped

,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.00,0.00,0.017544,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
1,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.066667,0.066667,0.066667,0.133333,0.2,0.066667,0.000000,0.000000,...,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
2,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.00,0.00,0.015625,0.000000,0.000000,0.015625,0.000000,0.00000,0.015625
3,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
4,Church and Wellesley,0.012987,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.012987,0.000000,...,0.012987,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.012987,0.00000,0.025974
5,"Commerce Court, Victoria Hotel",0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.040000,0.000000,...,0.000000,0.00,0.00,0.020000,0.000000,0.000000,0.010000,0.000000,0.00000,0.000000
6,"First Canadian Place, Underground city",0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.030000,0.000000,...,0.000000,0.00,0.01,0.010000,0.000000,0.000000,0.010000,0.000000,0.00000,0.000000
7,"Garden District, Ryerson",0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.00,0.00,0.000000,0.010000,0.010000,0.000000,0.000000,0.00000,0.000000
8,"Harbourfront East, Union Station, Toronto Islands",0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.00,0.01,0.010000,0.000000,0.000000,0.010000,0.000000,0.00000,0.000000
9,"Kensington Market, Chinatown, Grange Park",0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.00,0.00,0.033898,0.000000,0.050847,0.016949,0.000000,0.00000,0.000000


Let's print each neighborhood along with the top 5 most common venues.

In [25]:
num_top_venues = 5

for hood in downtown_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = downtown_grouped[downtown_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
          venue  freq
0   Coffee Shop  0.07
1  Cocktail Bar  0.05
2          Café  0.04
3    Restaurant  0.04
4        Bakery  0.04


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
              venue  freq
0   Airport Service  0.20
1    Airport Lounge  0.13
2       Coffee Shop  0.07
3             Plane  0.07
4  Sculpture Garden  0.07


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.17
1  Italian Restaurant  0.08
2                Café  0.05
3      Sandwich Place  0.05
4     Bubble Tea Shop  0.03


----Christie----
           venue  freq
0  Grocery Store  0.24
1           Café  0.18
2           Park  0.12
3          Diner  0.06
4     Restaurant  0.06


----Church and Wellesley----
                 venue  freq
0          Coffee Shop  0.08
1     Sushi Restaurant  0.05
2  Japanese Restaurant  0.05
3           Restaurant  0.04
4              Gay Bar  0.04


----Commerce C

#### Let's put that into a dataframe.

First, let's write a function to sort the venues in descending order.

In [26]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [27]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = downtown_grouped['Neighborhood']

for ind in np.arange(downtown_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(downtown_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Café,Cheese Shop,Seafood Restaurant,Restaurant,Bakery,Beer Bar,Eastern European Restaurant,Nightclub
1,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Coffee Shop,Harbor / Marina,Boat or Ferry,Bar,Sculpture Garden,Plane,Airport Terminal,Airport
2,Central Bay Street,Coffee Shop,Italian Restaurant,Sandwich Place,Café,Ice Cream Shop,Thai Restaurant,Bubble Tea Shop,Burger Joint,Salad Place,Japanese Restaurant
3,Christie,Grocery Store,Café,Park,Candy Store,Diner,Italian Restaurant,Restaurant,Baby Store,Athletics & Sports,Nightclub
4,Church and Wellesley,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Restaurant,Gay Bar,Yoga Studio,Café,Men's Store,Mediterranean Restaurant,Hotel


## 4. Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [28]:
# set number of clusters
kclusters = 5

downtown_grouped_clustering = downtown_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(downtown_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([4, 3, 0, 2, 4, 4, 4, 4, 4, 4], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [29]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

downtown_merged = downtown_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
downtown_merged = downtown_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

downtown_merged.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6543,-79.3606,4,Coffee Shop,Bakery,Park,Pub,Café,Theater,Breakfast Spot,Restaurant,Yoga Studio,Distribution Center
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6623,-79.3895,0,Coffee Shop,Yoga Studio,Creperie,Park,Mexican Restaurant,Italian Restaurant,Hobby Shop,Fried Chicken Joint,Distribution Center,Diner
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.6572,-79.3789,4,Clothing Store,Coffee Shop,Middle Eastern Restaurant,Café,Restaurant,Japanese Restaurant,Italian Restaurant,Cosmetics Shop,Bubble Tea Shop,Tea Room
3,M5C,Downtown Toronto,St. James Town,43.6515,-79.3754,4,Coffee Shop,Café,Cocktail Bar,Gastropub,Restaurant,American Restaurant,Theater,Italian Restaurant,Clothing Store,Gym
4,M5E,Downtown Toronto,Berczy Park,43.6448,-79.3733,4,Coffee Shop,Cocktail Bar,Café,Cheese Shop,Seafood Restaurant,Restaurant,Bakery,Beer Bar,Eastern European Restaurant,Nightclub


Finally, let's visualize the resulting clusters.

In [30]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(downtown_merged['Latitude'], downtown_merged['Longitude'], downtown_merged['Neighborhood'], downtown_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5. Examine Clusters

Cluster 1 (Red)

In [31]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 0, downtown_merged.columns[[2] + list(range(6, downtown_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,"Queen's Park, Ontario Provincial Government",Coffee Shop,Yoga Studio,Creperie,Park,Mexican Restaurant,Italian Restaurant,Hobby Shop,Fried Chicken Joint,Distribution Center,Diner
5,Central Bay Street,Coffee Shop,Italian Restaurant,Sandwich Place,Café,Ice Cream Shop,Thai Restaurant,Bubble Tea Shop,Burger Joint,Salad Place,Japanese Restaurant


Cluster 2 (Purple)

In [32]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 1, downtown_merged.columns[[2] + list(range(6, downtown_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Rosedale,Park,Playground,Trail,Creperie,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner


Cluster 3 (Blue)

In [33]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 2, downtown_merged.columns[[2] + list(range(6, downtown_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Christie,Grocery Store,Café,Park,Candy Store,Diner,Italian Restaurant,Restaurant,Baby Store,Athletics & Sports,Nightclub


Cluster 4 (Green)

In [34]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 3, downtown_merged.columns[[2] + list(range(6, downtown_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Coffee Shop,Harbor / Marina,Boat or Ferry,Bar,Sculpture Garden,Plane,Airport Terminal,Airport


Cluster 5 (Orange)

In [35]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 4, downtown_merged.columns[[2] + list(range(6, downtown_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Regent Park, Harbourfront",Coffee Shop,Bakery,Park,Pub,Café,Theater,Breakfast Spot,Restaurant,Yoga Studio,Distribution Center
2,"Garden District, Ryerson",Clothing Store,Coffee Shop,Middle Eastern Restaurant,Café,Restaurant,Japanese Restaurant,Italian Restaurant,Cosmetics Shop,Bubble Tea Shop,Tea Room
3,St. James Town,Coffee Shop,Café,Cocktail Bar,Gastropub,Restaurant,American Restaurant,Theater,Italian Restaurant,Clothing Store,Gym
4,Berczy Park,Coffee Shop,Cocktail Bar,Café,Cheese Shop,Seafood Restaurant,Restaurant,Bakery,Beer Bar,Eastern European Restaurant,Nightclub
7,"Richmond, Adelaide, King",Coffee Shop,Café,Restaurant,Gym,Deli / Bodega,Clothing Store,Thai Restaurant,Hotel,Salad Place,Pizza Place
8,"Harbourfront East, Union Station, Toronto Islands",Coffee Shop,Aquarium,Café,Hotel,Brewery,Restaurant,Sporting Goods Shop,Fried Chicken Joint,Italian Restaurant,Scenic Lookout
9,"Toronto Dominion Centre, Design Exchange",Coffee Shop,Café,Hotel,Italian Restaurant,American Restaurant,Salad Place,Seafood Restaurant,Japanese Restaurant,Restaurant,Sporting Goods Shop
10,"Commerce Court, Victoria Hotel",Coffee Shop,Restaurant,Café,Hotel,Gym,American Restaurant,Italian Restaurant,Deli / Bodega,Seafood Restaurant,Japanese Restaurant
11,"University of Toronto, Harbord",Café,Japanese Restaurant,Restaurant,Bar,Italian Restaurant,Bookstore,Bakery,Beer Bar,Beer Store,Sandwich Place
12,"Kensington Market, Chinatown, Grange Park",Café,Mexican Restaurant,Vietnamese Restaurant,Coffee Shop,Bakery,Grocery Store,Gaming Cafe,Vegetarian / Vegan Restaurant,Bar,Dessert Shop
